# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [21]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from sklearn.tree import DecisionTreeRegressor
from sklearn import tree
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split

import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.metrics import r2_score



In [2]:
df = pd.read_csv('previsao_de_renda.csv')
df = df.dropna()

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12427 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             12427 non-null  int64  
 1   data_ref               12427 non-null  object 
 2   id_cliente             12427 non-null  int64  
 3   sexo                   12427 non-null  object 
 4   posse_de_veiculo       12427 non-null  bool   
 5   posse_de_imovel        12427 non-null  bool   
 6   qtd_filhos             12427 non-null  int64  
 7   tipo_renda             12427 non-null  object 
 8   educacao               12427 non-null  object 
 9   estado_civil           12427 non-null  object 
 10  tipo_residencia        12427 non-null  object 
 11  idade                  12427 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  12427 non-null  float64
 14  renda                  12427 non-null  float64
dtypes:

1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

# 1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento)

In [4]:
# Definindo as variáveis independentes (X) e a variável dependente (y)
X = df.drop(['Unnamed: 0', 'data_ref', 'id_cliente'], axis=1)  # Excluí colunas desnecessárias
y = np.log(df['renda'])  # Aplicando o logaritmo na coluna 'renda'

# Dividindo os dados em treinamento e teste (75% para treinamento, 25% para teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=100)

In [5]:
print("X_train = ", X_train.shape, "y_train = ", y_train.shape)
print("X_test = ", X_test.shape, "y_test = ", y_test.shape)

X_train =  (9320, 12) y_train =  (9320,)
X_test =  (3107, 12) y_test =  (3107,)


# 2. Rode uma regularização ridge com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o na base de testes. Qual o melhor modelo?

In [6]:
X_train_patsy = patsy.dmatrices('''np.log(renda) ~ C(sexo) 
                    + C(posse_de_veiculo) 
                    + C(posse_de_imovel)
                    + qtd_filhos 
                    + C(tipo_renda) 
                    + C(educacao, Treatment(2)) 
                    + C(estado_civil) 
                    + C(tipo_residencia, Treatment(1)) 
                    + idade
                    + tempo_emprego
                    + qt_pessoas_residencia 
                    + 1''', X_train)

X_test_patsy = patsy.dmatrices('''np.log(renda) ~ C(sexo) 
                    + C(posse_de_veiculo) 
                    + C(posse_de_imovel)
                    + qtd_filhos 
                    + C(tipo_renda) 
                    + C(educacao, Treatment(2)) 
                    + C(estado_civil) 
                    + C(tipo_residencia, Treatment(1)) 
                    + idade
                    + tempo_emprego
                    + qt_pessoas_residencia 
                    + 1''', X_test)


In [7]:
# Lista de valores alpha
alpha_values = [0, 0.001, 0.005, 0.01, 0.05, 0.1]

# Inicializa variáveis para armazenar resultados
melhor_modelo = None
melhor_alpha = None
melhor_r2 = -float('inf')

# Itera sobre os valores alpha
for alpha in alpha_values:
    # Cria e treina o modelo Ridge com o valor específico de alpha
    modelo_ridge = smf.ols(X_train_patsy, data = X_train).fit_regularized(method = 'elastic_net' 
                         , refit = True
                         , L1_wt = 0
                         , alpha = alpha)

    # Faça previsões na base de testes
    y_pred = modelo_ridge.predict(X_test)

    # Avalie o desempenho usando o R²
    r2 = r2_score(y_test, y_pred)

    # Imprime os resultados para cada valor de alpha
    print(f"Alpha: {alpha}, R²: {r2:.5f}")

    # Atualiza o melhor modelo se necessário
    if r2 > melhor_r2:
        melhor_modelo = modelo_ridge
        melhor_alpha = alpha
        melhor_r2 = r2

print(f"\nMelhor modelo:\nAlpha: {melhor_alpha}, R²: {melhor_r2:.5f}")




Alpha: 0, R²: 0.36592
Alpha: 0.001, R²: 0.36593
Alpha: 0.005, R²: 0.35327
Alpha: 0.01, R²: 0.32241
Alpha: 0.05, R²: 0.01001
Alpha: 0.1, R²: -0.28187

Melhor modelo:
Alpha: 0.001, R²: 0.36593


In [8]:
modelo_ridge_com_melhor_alpha = smf.ols(X_train_patsy, data = X_train).fit_regularized(method = 'elastic_net' 
                         , refit = True
                         , L1_wt = 0
                         , alpha = 0.001)

# 3.Faça o mesmo que no passo 2, com uma regressão LASSO. Qual método chega a um melhor resultado?

In [9]:
# Lista de valores alpha
alpha_values = [0, 0.001, 0.005, 0.01, 0.05, 0.1]

# Inicializa variáveis para armazenar resultados
melhor_modelo_lasso = None
melhor_alpha_lasso = None
melhor_r2_lasso = -float('inf')

# Itera sobre os valores alpha
for alpha in alpha_values:
    # Cria e treina o modelo Ridge com o valor específico de alpha
    modelo_lasso = smf.ols(X_train_patsy, data = X_train).fit_regularized(method = 'elastic_net' 
                         , refit = True
                         , L1_wt = 1
                         , alpha = alpha)

    # Faça previsões na base de testes
    y_pred = modelo_lasso.predict(X_test)

    # Avalie o desempenho usando o R²
    r2 = r2_score(y_test, y_pred)

    # Imprime os resultados para cada valor de alpha
    print(f"Alpha: {alpha}, R²: {r2:.5f}")

    # Atualiza o melhor modelo se necessário
    if r2 > melhor_r2_lasso:
        melhor_modelo_lasso = modelo_lasso
        melhor_alpha_lasso = alpha
        melhor_r2_lasso = r2

print(f"\nMelhor modelo LASSO:\nAlpha: {melhor_alpha_lasso}, R²: {melhor_r2_lasso:.5f}")

Alpha: 0, R²: 0.36592
Alpha: 0.001, R²: 0.36191
Alpha: 0.005, R²: 0.35279
Alpha: 0.01, R²: 0.34768
Alpha: 0.05, R²: 0.34768
Alpha: 0.1, R²: 0.34768

Melhor modelo LASSO:
Alpha: 0, R²: 0.36592


Para uma regularização ridge o melhor Alpha: 0.001 que resulta no melhor R²: 0.36593, ja com a regularização LASSO temos Alpha: 0, R²: 0.36592. Observa-se que no LASSO o melhor alpha foi um valor menor que o do melhor alplha ridge. e percebemos que não uma diferença siginficativa entre os R²

In [10]:
modelo_lasso_com_melhor_alpha = smf.ols(X_train_patsy, data = X_train).fit_regularized(method = 'elastic_net' 
                         , refit = True
                         , L1_wt = 1
                         , alpha = 0)

# 4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?

In [11]:
#df = df.drop(['Unnamed: 0', 'data_ref', 'id_cliente'], axis=1)  # Excluí colunas desnecessárias
# Identifique as colunas categóricas do tipo 'object' e 'bool'
colunas_categoricas = df.select_dtypes(include=['object', 'bool']).columns

# Crie o DataFrame de dummies para as colunas 'object'
df_dummies_obj = pd.get_dummies(df[colunas_categoricas.drop(df.select_dtypes('bool').columns)], drop_first=True)

# Crie o DataFrame de dummies para as colunas 'bool'
df_dummies_bool = df[colunas_categoricas.intersection(df.select_dtypes('bool').columns)].astype(int)

# Concatene ambos os DataFrames
df_final = pd.concat([df.drop(colunas_categoricas, axis=1), df_dummies_obj, df_dummies_bool], axis=1)
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12427 entries, 0 to 14999
Data columns (total 41 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Unnamed: 0                     12427 non-null  int64  
 1   id_cliente                     12427 non-null  int64  
 2   qtd_filhos                     12427 non-null  int64  
 3   idade                          12427 non-null  int64  
 4   tempo_emprego                  12427 non-null  float64
 5   qt_pessoas_residencia          12427 non-null  float64
 6   renda                          12427 non-null  float64
 7   data_ref_2015-02-01            12427 non-null  uint8  
 8   data_ref_2015-03-01            12427 non-null  uint8  
 9   data_ref_2015-04-01            12427 non-null  uint8  
 10  data_ref_2015-05-01            12427 non-null  uint8  
 11  data_ref_2015-06-01            12427 non-null  uint8  
 12  data_ref_2015-07-01            12427 non-null 

In [12]:
# Definindo as variáveis independentes (X) e a variável dependente (y)
X_s = df_final.drop('Unnamed: 0', axis=1)
y_s = np.log(df_final['renda'])  # Aplicando o logaritmo na coluna 'renda'

# Dividindo os dados em treinamento e teste (75% para treinamento, 25% para teste)
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X_s, y_s, train_size=0.75, random_state=100)

In [13]:
print("X_train_s = ", X_train_s.shape, "y_train_s = ", y_train.shape)
print("X_test_s = ", X_test_s.shape, "y_test_s = ", y_test.shape)

X_train_s =  (9320, 40) y_train_s =  (9320,)
X_test_s =  (3107, 40) y_test_s =  (3107,)


In [14]:
# Função para realizar o modelo stepwise
def stepwise_selection(X, y, initial_list=[], threshold_in=0.01, threshold_out=0.05, verbose=True):
    included = list(initial_list)
    while True:
        changed = False
        # Adiciona variáveis que não estão na lista e têm um p-valor abaixo do limiar de entrada
        excluded = list(set(X.columns) - set(included))
        # Especifique explicitamente o tipo de dados da Série como float64
        new_pval = pd.Series(index=excluded, dtype=float)
        for new_column in excluded:
            # Adicione um print para verificar os tipos de dados antes de ajustar o modelo
            print(f"Tipo de dado de {new_column}: {X[new_column].dtype}")
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included + [new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed = True
            if verbose:
                print(f'Adicionando variável: {best_feature}  p-value: {best_pval:.4f}')
        
        # Remove variáveis que têm um p-valor acima do limiar de saída
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max()
        if worst_pval > threshold_out:
            changed = True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print(f'Removendo variável: {worst_feature}  p-value: {worst_pval:.4f}')
        
        if not changed:
            break
    return included

In [15]:
# Realiza o modelo stepwise no conjunto de treinamento
selected_features = stepwise_selection(X_train_s, y_train_s)

Tipo de dado de tipo_renda_Bolsista: uint8
Tipo de dado de educacao_Superior completo: uint8
Tipo de dado de data_ref_2016-01-01: uint8
Tipo de dado de tipo_residencia_Governamental: uint8
Tipo de dado de estado_civil_Viúvo: uint8
Tipo de dado de tipo_residencia_Comunitário: uint8
Tipo de dado de educacao_Secundário: uint8
Tipo de dado de estado_civil_Separado: uint8
Tipo de dado de idade: int64
Tipo de dado de data_ref_2015-06-01: uint8
Tipo de dado de posse_de_veiculo: int32
Tipo de dado de qtd_filhos: int64
Tipo de dado de data_ref_2015-05-01: uint8
Tipo de dado de data_ref_2015-12-01: uint8
Tipo de dado de data_ref_2015-02-01: uint8
Tipo de dado de data_ref_2015-11-01: uint8
Tipo de dado de data_ref_2016-03-01: uint8
Tipo de dado de data_ref_2015-09-01: uint8
Tipo de dado de tipo_renda_Pensionista: uint8
Tipo de dado de educacao_Superior incompleto: uint8
Tipo de dado de data_ref_2015-07-01: uint8
Tipo de dado de tempo_emprego: float64
Tipo de dado de qt_pessoas_residencia: float64

Tipo de dado de estado_civil_Separado: uint8
Tipo de dado de idade: int64
Tipo de dado de data_ref_2015-06-01: uint8
Tipo de dado de posse_de_veiculo: int32
Tipo de dado de qtd_filhos: int64
Tipo de dado de data_ref_2015-05-01: uint8
Tipo de dado de data_ref_2015-12-01: uint8
Tipo de dado de data_ref_2015-02-01: uint8
Tipo de dado de data_ref_2015-11-01: uint8
Tipo de dado de data_ref_2016-03-01: uint8
Tipo de dado de data_ref_2015-09-01: uint8
Tipo de dado de tipo_renda_Pensionista: uint8
Tipo de dado de educacao_Superior incompleto: uint8
Tipo de dado de data_ref_2015-07-01: uint8
Tipo de dado de qt_pessoas_residencia: float64
Tipo de dado de data_ref_2015-08-01: uint8
Tipo de dado de tipo_residencia_Estúdio: uint8
Tipo de dado de data_ref_2015-04-01: uint8
Tipo de dado de id_cliente: int64
Tipo de dado de tipo_residencia_Com os pais: uint8
Tipo de dado de posse_de_imovel: int32
Tipo de dado de data_ref_2016-02-01: uint8
Tipo de dado de estado_civil_Solteiro: uint8
Tipo de dado de ti

In [16]:
# Ajustando o modelo final usando as variáveis selecionadas
stepwise_model = sm.OLS(y_train_s, sm.add_constant(X_train_s[selected_features])).fit()

In [17]:
# Fazendo as previsões na base de testes
y_pred_s = stepwise_model.predict(sm.add_constant(X_test_s[selected_features]))

In [18]:
# Avaliando o desempenho usando R² na base de testes
r2_teste = r2_score(y_test_s, y_pred_s)

In [19]:
print(f"\nR² na base de testes usando modelo stepwise: {r2_teste:.4f}")
print(f"Variáveis selecionadas: {selected_features}")



R² na base de testes usando modelo stepwise: 0.6499
Variáveis selecionadas: ['tempo_emprego', 'sexo_M', 'renda', 'tipo_renda_Empresário', 'educacao_Superior completo', 'posse_de_imovel', 'idade', 'tipo_renda_Servidor público']


O melhor valor R² foi obtido com o modelo stepwise, pois para uma regularização ridge obtivemos R²: 0.36593, e com a regularização LASSO temos R²: 0.36592.

Assim o melhor modelo é resultado é com o MODELO STEPWISE

# 5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?

In [22]:
def extract_model_metrics(model, X, y, name):
    # Calcule as previsões
    y_pred = model.predict(X)
    
    # Calcule R²
    r2 = r2_score(y, y_pred)
    
    # Calcule R² ajustado
    n = len(y)
    k = X.shape[1]  # número de variáveis independentes
    adj_r2 = 1 - (1 - r2) * (n - 1) / (n - k - 1)
    
    # Calcule AIC
    residuals = y - y_pred
    sse = np.sum(residuals**2)
    aic = n * np.log(sse/n) + 2 * k
    
    # Calcule BIC
    bic = n * np.log(sse/n) + k * np.log(n)
    
    # Crie um DataFrame com as métricas
    metrics_df = pd.DataFrame({
        'Model': [name],
        'R-squared': [r2],
        'Adj. R-squared': [adj_r2],
        'AIC': [aic],
        'BIC': [bic]
    })
    
    return metrics_df


In [23]:
#Extraindo as métricas dos modelos gerados

# Obtenha as colunas usadas durante o treinamento do modelos
colunas_usadas_stepwise = stepwise_model.model.exog_names
ridge_colunas_usadas = X_train.columns
lasso_colunas_usadas = X_train.columns

# Adicionando uma coluna "const" aos dados de teste
X_test_s_stepwise = sm.add_constant(X_test_s[colunas_usadas_stepwise[1:]])
X_test_ridge = sm.add_constant(X_test[ridge_colunas_usadas])
X_test_lasso = sm.add_constant(X_test[lasso_colunas_usadas])

# Extraindo as métricas para o modelos
stepwise_metrics = extract_model_metrics(stepwise_model, X_test_s_stepwise, y_test_s, 'Stepwise')
ridge_metrics = extract_model_metrics(modelo_ridge_com_melhor_alpha, X_test_ridge, y_test, 'Ridge')
lasso_metrics = extract_model_metrics(modelo_lasso_com_melhor_alpha, X_test_lasso, y_test, 'LASSO')

#Gerando o DataFrame compativo das méticas
df_comparativo = pd.concat([stepwise_metrics, ridge_metrics, lasso_metrics], ignore_index=True)

df_comparativo

,Model,R-squared,Adj. R-squared,AIC,BIC
0,Stepwise,0.649869,0.648852,-3896.363906,-3841.991190
1,Ridge,0.365930,0.363265,-2043.265383,-1964.727015
2,LASSO,0.365916,0.363251,-2043.196228,-1964.657860


O modelo Stepwise tem o R-squared e o Adj. R-squared mais altos, indicando que ele explica melhor a variação nos dados.

Os modelos Ridge e LASSO têm valores mais baixos para R-squared e Adj. R-squared, sugerindo um ajuste menos preciso aos dados em comparação ao modelo Stepwise.

Quanto ao AIC e BIC, os valores mais baixos indicam melhor ajuste. Nesse caso, Ridge e LASSO têm valores mais baixos do que o modelo Stepwise, sugerindo que, considerando a penalidade por complexidade, os modelos de regularização têm melhor desempenho.

Conclusão:

A escolha do "melhor" modelo depende dos objetivos específicos do seu problema. Se o objetivo é maximizar a explicação da variância nos dados, o modelo Stepwise pode ser preferido. Se o objetivo é evitar overfitting e encontrar um equilíbrio entre ajuste e simplicidade, Ridge e LASSO podem ser escolhas melhores com base nos valores mais baixos de AIC e BIC.

# 6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.

In [24]:
modelo_modificado = smf.ols('np.log(renda) ~  tempo_emprego + sexo_M + idade + qt_pessoas_residencia', data = X_train_s).fit()
modelo_modificado.summary()



<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.342
Model:                            OLS   Adj. R-squared:                  0.341
Method:                 Least Squares   F-statistic:                     1208.
Date:                Fri, 19 Jan 2024   Prob (F-statistic):               0.00
Time:                        13:48:42   Log-Likelihood:                -10287.
No. Observations:                9320   AIC:                         2.058e+04
Df Residuals:                    9315   BIC:                         2.062e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                 7.2762      0.044    164.236      0.000       7.189       7.363
tempo_emprego             0.0607      0.001     50.932      0.000       0.058       0.063
sexo_M                    0.7828      0.016     48.934      0.000       0.751       0.814
idade                     0.0051      0.001      5.777      0.000       0.003       0.007
qt_pessoas_residencia     0.0125      0.008      1.504      0.133      -0.004       0.029
==============================================================================
Omnibus:                        0.323   Durbin-Watson:                   2.019
Prob(Omnibus):                  0.851   Jarque-Bera (JB):                0.289
Skew:                           0.004   Prob(JB):                        0.865
Kurtosis:                       3.026   Cond. No.                         252.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [25]:
modelo_modificado2 = smf.ols('np.log(renda) ~  tempo_emprego + I(tempo_emprego**2) + sexo_M + idade + qt_pessoas_residencia', data = X_train_s).fit()
modelo_modificado2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.345
Model:                            OLS   Adj. R-squared:                  0.344
Method:                 Least Squares   F-statistic:                     980.2
Date:                Fri, 19 Jan 2024   Prob (F-statistic):               0.00
Time:                        13:48:45   Log-Likelihood:                -10264.
No. Observations:                9320   AIC:                         2.054e+04
Df Residuals:                    9314   BIC:                         2.058e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                 7.2010      0.046    157.999      0.000       7.112       7.290
tempo_emprego             0.0791      0.003     26.632      0.000       0.073       0.085
I(tempo_emprego ** 2)    -0.0007      0.000     -6.764      0.000      -0.001      -0.000
sexo_M                    0.7828      0.016     49.052      0.000       0.752       0.814
idade                     0.0054      0.001      6.195      0.000       0.004       0.007
qt_pessoas_residencia     0.0086      0.008      1.032      0.302      -0.008       0.025
==============================================================================
Omnibus:                        0.293   Durbin-Watson:                   2.022
Prob(Omnibus):                  0.864   Jarque-Bera (JB):                0.267
Skew:                           0.010   Prob(JB):                        0.875
Kurtosis:                       3.018   Cond. No.                     1.36e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.36e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Mesmo aplicando transformação nas variáveis, não se obtem um R² melhor

# 7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [26]:
tree_reg = DecisionTreeRegressor(max_depth=4, min_samples_leaf=10, random_state=42)
tree_reg.fit(X_train_s, y_train_s)

DecisionTreeRegressor(max_depth=4, min_samples_leaf=10, random_state=42)

In [28]:
y_pred = tree_reg.predict(X_test_s)


In [29]:
r2 = r2_score(y_test, y_pred)
print(f"R²: {r2}")

R²: 0.9904040601075268


Com Árvore de Regressão foi possível obter o melhor R² -> 0.99